In [4]:
### AEBagging model SNR
import numpy as np 
import matplotlib.pyplot as plt 
from matplotlib.gridspec import GridSpec 
from matplotlib.collections import LineCollection
from matplotlib.gridspec import GridSpec
import matplotlib.gridspec as gridspec 
import scipy.signal as signal 
import os 
from matplotlib.pyplot import MultipleLocator 
from matplotlib.ticker import FuncFormatter
plt.rcParams['figure.figsize'] = (16, 12)
plt.rcParams['figure.dpi'] = 150
plt.rcParams['font.size'] = 12
plt.switch_backend("agg") 
# plt.rcParams['font.sans-serif'] = 'SimHei'
# plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['ytick.major.width'] = 2
plt.rcParams['xtick.major.size']  = 8
plt.rcParams['xtick.minor.size']  = 4
plt.rcParams['ytick.major.size']  = 8
plt.rcParams['ytick.minor.size']  = 4

### 如果UNet++报错，换绝对路径
files = open("test_results\AEbagging.txt", "r") 
phase_list = [[aa for aa in itr.strip().split(',') if len(aa)>0] for itr in files.readlines()] 
files.close()

phase_head = []
for itr in phase_list:
    if "#phase" in itr[0] or "#none" in itr[0]:
        phase_head.append(0) 
    else:
        phase_head.append(1)
phase_head = np.array(phase_head)
phase_list = np.array(phase_list, dtype=object)
split_idx = np.where(phase_head==0)[0][1:] 
phase_list = np.split(phase_list, split_idx) 

tp = [0 for itr in range(1)]  ##[0,0]
fp = [0 for itr in range(1)]  ##[0,0]
fn = [0 for itr in range(1)]  ##[0,0]
std = [[] for itr in range(1)]  ##[[],[]]

gap1=100
gap2=600

p1 =[]
r1 =[]
f1 =[]
error_aves=[]
error_stds=[]
for i in range(6):
    tp = [0 for itr in range(1)]  ##[0,0]
    fp = [0 for itr in range(1)]  ##[0,0]
    fn = [0 for itr in range(1)]  ##[0,0]
    for phase in phase_list:
        if float(phase[0][2])>10*i and float(phase[0][2])<=10*(i+1) :
            phases = phase
        else:
            continue
    
        if len(phases)<=1:
            fn[0] +=1
            continue
        else:
            if "#none" not in phases[0][0]:
                mm_time = float(phases[0][1])  #真实值
        allpick = [False]
        erro=[]
        for nn_pick in phases[1:]:
            nn_time = float(nn_pick[0])  #模型拾取
            erro.append(np.abs(mm_time - nn_time))
        error = min(erro)
        index = np.argmin(erro)
        nn_times = float(phases[1:][index][0])
        if error < gap1:
            tp[0] += 1
            allpick[0] = True
        else:
            fp[0] += 1
    
        # 标准差 均值
        if error < gap2: 
            std[0].append((mm_time-nn_times)/3000)
        for tps in range(len(tp)):
            if allpick[tps] == False:
                fn[tps] +=1
    P = [0 for itr in range(1)]
    R = [0 for itr in range(1)]
    for tps in range(1):
        P[tps] = tp[tps]/(tp[tps]+fp[tps]+1e-9)
        R[tps] = tp[tps]/(tp[tps]+fn[tps]+1e-9)
    F1=(2*P[0]*R[0])/(P[0]+R[0]+1e-6)
    p1.append(P[0])
    r1.append(R[0])
    f1.append(F1)
    
    error_ave=np.mean(std)
    error_std=np.std(std)
    
    error_aves.append(error_ave)
    error_stds.append(error_std)

error_aves=np.array(error_aves)
p1=np.array(p1)
r1=np.array(r1)
f1=np.array(f1)
sum=np.vstack((p1,r1,f1))

labels = ['4~10', '10~20', '20~30', '30~40',"40~50", "50~60"]
x = np.arange(len(labels))
width = 0.18

colors = ["#ff0000", "#0000ff", "#843C0C", "#1F4E79"]
fig, ax = plt.subplots(figsize=(9,6),dpi = 100)
rects1 = ax.bar(x-width/2 , sum[0], width-0.03,  zorder=2, color="C0", alpha=0.95, label="Precision") #(12.58/0.1)edgecolor='k', 
rects2 = ax.bar(x+width/2, sum[1], width-0.03, zorder=2, color="C1",alpha=0.95, label="Recall") #(4.78/0.1)
rects3 = ax.bar(x + 3*width/2, sum[2], width-0.03, zorder=2, color="C2",alpha=0.95, label="F1-score") #(248.7/0.1)

y_major_locator = MultipleLocator(0.1)
ax.yaxis.set_major_locator(y_major_locator)
ax.set_xlabel("SNR (dB)", fontsize=24) 
# ax.set_ylabel('Ratio', fontsize=24)        
ax.set_xticks(x+0.09, labels)
# ax.legend(fontsize=18)
ax.tick_params(axis='x', labelsize=22)
ax.tick_params(axis='y', labelsize=22)
ax.set_ylim(0.7,1.000)
ax.grid(which='major', axis='y', linewidth=0.75, linestyle='-', color='0.75', zorder=0)
ax.grid(which='minor', axis='y', linewidth=0.25, linestyle='-', color='0.75', zorder=0)

ax.tick_params(bottom=False, top=False, left=False, right=False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
# ax.legend(ncol=3, fontsize=20, labelcolor="k", loc=(0.105,1.00), frameon=False)

plt.tight_layout()
plt.savefig('model_SNR.png', dpi=600)


################ 专门用于UNet++
# y_major_locator = MultipleLocator(0.2)
# ax.yaxis.set_major_locator(y_major_locator)
# ax.set_xlabel("SNR (dB)", fontsize=24) 
# # ax.set_ylabel('Ratio', fontsize=24)        
# ax.set_xticks(x+0.09, labels)
# # ax.legend(fontsize=18)
# ax.tick_params(axis='x', labelsize=22)
# ax.tick_params(axis='y', labelsize=22)
# ax.set_ylim(0.0,1.002)
# ax.grid(which='major', axis='y', linewidth=0.75, linestyle='-', color='0.75', zorder=0)
# ax.grid(which='minor', axis='y', linewidth=0.25, linestyle='-', color='0.75', zorder=0)

# ax.tick_params(bottom=False, top=False, left=False, right=False)
# ax.spines['top'].set_visible(False)
# ax.spines['bottom'].set_visible(False)
# ax.spines['left'].set_visible(False)
# ax.spines['right'].set_visible(False)
# # ax.legend(ncol=3, fontsize=20, labelcolor="k", loc=(0.105,1.00), frameon=False)

# plt.tight_layout()
# plt.savefig('Unet++_SNR.png', dpi=600, format='png')

In [5]:
### AEBagging model error
import numpy as np 
import matplotlib.pyplot as plt 
from matplotlib.gridspec import GridSpec 
from matplotlib.collections import LineCollection
from matplotlib.gridspec import GridSpec
import matplotlib.gridspec as gridspec 
import scipy.signal as signal 
import os 
from matplotlib.pyplot import MultipleLocator 
from matplotlib.ticker import FuncFormatter
plt.rcParams['figure.figsize'] = (16, 12)
plt.rcParams['figure.dpi'] = 150
plt.rcParams['font.size'] = 12
plt.switch_backend("agg") 
plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['axes.unicode_minus'] = False
# plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['ytick.major.width'] = 2
plt.rcParams['xtick.major.size']  = 8
plt.rcParams['xtick.minor.size']  = 4
plt.rcParams['ytick.major.size']  = 8
plt.rcParams['ytick.minor.size']  = 4

files = open("test_results\AEbagging.txt", "r") 
# files = open(r"C:\Users\Administrator.DESKTOP-Q5G5OQ4\Desktop\AEbagging\test_results\UNet++\UNet++.txt", "r") 
phase_list = [[aa for aa in itr.strip().split(',') if len(aa)>0] for itr in files.readlines()] 
files.close()

phase_head = []
for itr in phase_list:
    if "#phase" in itr[0] or "#none" in itr[0]:
        phase_head.append(0) 
    else:
        phase_head.append(1)
phase_head = np.array(phase_head) 
phase_list = np.array(phase_list, dtype=object)
split_idx = np.where(phase_head==0)[0][1:] 
phase_list = np.split(phase_list, split_idx) 

tp = [0 for itr in range(1)]  ##[0,0]
fp = [0 for itr in range(1)]  ##[0,0]
fn = [0 for itr in range(1)]  ##[0,0]
std = [[] for itr in range(1)]  ##[[],[]]

gap1=100
gap2=600
for phases in phase_list:
    if len(phases)<=1:
        fn[0] +=1
        continue
    else:
        if "#none" not in phases[0][0]:
            mm_time = float(phases[0][1])  #真实值
    allpick = [False]
    erro=[]
    for nn_pick in phases[1:]:
        nn_time = float(nn_pick[0])
        erro.append(np.abs(mm_time - nn_time))
    error = min(erro)
    index = np.argmin(erro)
    nn_times = float(phases[1:][index][0])
    if error < gap1:
        tp[0] += 1
        allpick[0] = True 
    else:
        fp[0] += 1
    if error < gap2: 
        std[0].append((mm_time-nn_times)/3000)
    for tps in range(len(tp)):
        if allpick[tps] == False:
            fn[tps] += 1
            
P = [0 for itr in range(1)]
R = [0 for itr in range(1)]
M = [0 for itr in range(1)]
for tps in range(1):
    P[tps] = tp[tps]/(tp[tps]+fp[tps]+1e-9)
    R[tps] = tp[tps]/(tp[tps]+fn[tps]+1e-9)
    M[tps] = fn[tps]/(tp[tps]+fn[tps]+1e-9)

F1=(2*P[0]*R[0])/(P[0]+R[0])

print("样本数量:", len(phase_list), "查准率:", P[0], "查全率:", R[0], "P与R的调和平均值:", F1, "误拾率:",M[0])
stds=np.array(std[0])*1000

mae=np.sum(np.abs(std[0]))/len(std[0])*1000
error_ave=np.mean(std)*1000
error_std=np.std(std)*1000
print(error_ave, error_std,mae)

colors = ["#ff0000", "#0000ff", "w"]
model="AEbagging"
Phase="P"
fig, ax = plt.subplots(1,1, figsize=(8.5,8), dpi=100)
n,bins,pathes=ax.hist(stds, bins=32, color=colors[0], alpha=0.5, edgecolor="None")
ax.hist(stds, bins=32, ls="-", facecolor="None", edgecolor="k", lw=1.0)
ax.set_xlim([-150, 150])
ax.set_ylim([0, 1400])
ax.set_title(f"M:{model}\nPhase:{Phase}\nP={P[0]:.3f}\nR={R[tps]:.3f}\nF1={F1:.3f}\nMAE={mae:.1f}μs\n$\mu$={error_ave:.1f}μs\n$\sigma$={error_std:.1f}μs", fontsize=25, x=0.03, y=0.95, ha="left", va="top")
ax.set_title(f"M:{model}\nPhase:{Phase}\nP={P[0]:.3f}\nR={R[tps]:.3f}\nF1={F1:.3f}\n$\mu$={error_ave:.1f} μs\n$\sigma$={error_std:.1f} μs", fontsize=25, x=0.03, y=0.95, ha="left", va="top")
ax.tick_params(axis='both', labelsize=32)
ax.set_xlabel('Time Residuals (μs)', fontsize=34)
ax.set_ylabel('Number of Picks', fontsize=34)
y_major_locator=MultipleLocator(200)
ax.yaxis.set_major_locator(y_major_locator)
x_major_locator=MultipleLocator(50)
ax.xaxis.set_major_locator(x_major_locator)
# ax.minorticks_on()
# plt.savefig("model_error.png", bbox_inches="tight", dpi=600)

样本数量: 2050 查准率: 0.9266862170083447 查全率: 0.9248780487800367 P与R的调和平均值: 0.9257812499995479 误拾率: 0.07512195121947556
-0.6198198198198198 20.88989087005185 9.991809991809992
